## Static Routes and Live OBA Data Overlaid

Combine two previous experiments generating route polyines and live position data onto a single map.

In [24]:
# Bootstrap OBA API KEY

import os
import pandas as pd
import requests

from dotenv import load_dotenv

# Get OBA API KEY
load_dotenv()

ROUTES = {
    8: 100275,
    48: 100228,
    62: 100252,
    153: 100048,
}

ROUTE_ID = ROUTES[62]

# SETUP DATAFRAMES FOR GTFS STATIC FILES

# REF: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html#copy-on-write-optimizations
# Used based on warning when attempting to update a subset of a dataframe downstream.
pd.set_option('mode.copy_on_write', True)
pd.options.mode.copy_on_write = True

AGENCY_DIR = 'kc_metro'

data = {
    "calendar": {
        "csv": "calendar.txt",
        "df": None
    },
    "routes": {
        "csv": "routes.txt",
        "df": None
    },
    "stops": {
        "csv": "stops.txt",
        "df": None
    },
    "stop_times": {
        "csv": "stop_times.txt",
        "df": None
    },
    "trips": {
        "csv": "trips.txt",
        "df": None
    },
}

for header, vals in data.items():
    data_dir = os.path.join(os.getcwd(), os.pardir, os.pardir, 'data')
    data_csv =  os.path.join(data_dir, AGENCY_DIR, vals['csv'])
    data_df = pd.read_csv(data_csv)
    data[header]['df'] = data_df


## Get Coordinates For Route Polyine From Static GTFS Files

In [25]:
# Get the first trip_id for our route of interest
# From `trips.txt`
trips_df = data['trips']['df']
trips_filtered_by_route = trips_df[trips_df['route_id'] == ROUTE_ID]['trip_id']
trips_filtered_by_route.head()

FIRST_TRIP_ID = trips_filtered_by_route.head(1).values[0]

# Filter `stop_times.txt` by trip_id to get stop_ids for a route.
stop_times_df = data['stop_times']['df']
stop_ids = stop_times_df[stop_times_df['trip_id'] == FIRST_TRIP_ID]['stop_id']

# Join previously filtered stop_ids to stops.txt
# giving us the stop geometries
stops_df = data['stops']['df']
stops_filtered_by_route = pd.merge(stop_ids, stops_df)

## `Polyline()` Expects a list of coordinate tuples.
route_coord_data = list(zip(stops_filtered_by_route['stop_lat'], stops_filtered_by_route['stop_lon']))

## Query OBA API For Live Bus Positions

In [26]:
# CONSTS for API Calls

API_KEY = os.getenv("API_KEY")
API_ROOT_URL = "https://api.pugetsound.onebusaway.org/api/where"
ENDPOINT_TRIPS_FOR_ROUTE = "trips-for-route"

"""
Returns a response object from the OBA API for the trips-for-route endpoint
REF: https://developer.onebusaway.org/api/where/methods/trips-for-route

include_schedule: boolean. Set this to `True` to include detailed
schedule information for the route.
"""
def query_trips_for_route(route_id, include_schedule=False):
    request_url = f'{API_ROOT_URL}/{ENDPOINT_TRIPS_FOR_ROUTE}/1_{ROUTE_ID}.json'
    response_data = requests.get(request_url, params={"key": API_KEY, "includeSchedule": include_schedule})
    print(response_data.url)
    return response_data.json()

## Make the Map

In [27]:

import folium

m = folium.Map(location=[47.617, -122.34], zoom_start=13)

oba_live_trip_data = query_trips_for_route(ROUTE_ID)

def generate_folium_marker_for_trip(trip_element):
    kw = {"prefix": "fa", "color": "green", "icon": "arrow-up"}
    angle = int(trip_element["status"]["orientation"])
    icon = folium.Icon(angle=angle, **kw)

    return folium.Marker(
        location=[trip_element["status"]["position"]["lat"], trip_element["status"]["position"]["lon"]],
        tooltip=trip_element["status"]["vehicleId"],
        popup=trip_element["status"]["phase"],
        icon=icon,
    )

# Add the route polyline to the map
folium.PolyLine(route_coord_data).add_to(m)

# Generate markers showing bus location for each trip,
# skipping trips that don't have a "status" key
markers = [generate_folium_marker_for_trip(trip) for trip in oba_live_trip_data["data"]["list"] if "status" in trip.keys()]
[marker.add_to(m) for marker in markers]
# Display the map
m

https://api.pugetsound.onebusaway.org/api/where/trips-for-route/1_100252.json?key=772e8f7d-77d8-4c54-8e20-4630a03a1126&includeSchedule=False
